In [1]:
import pandas as pd

from bs4 import BeautifulSoup

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from collections import defaultdict

In [3]:
from gensim import models
import numpy as np

In [4]:
import gensim
from gensim import corpora
from pprint import pprint
import string

In [5]:
df = pd.read_csv(r"D:\data\openparliament\statements_nospeaker_en.csv")

C:\Users\alecr\Anaconda3\envs\gensim_env\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (20,23,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df.drop('Unnamed: 0',axis=1, inplace=True)

In [7]:
df.head() 

,id,politician_id,riding_id,party_id,end_date,start_date,id.1,document_id,time,h1_en,...,slug,urlcache,h1_fr,h2_fr,h3_fr,who_fr,who_context_fr,wordcount_en,tokenized_content,slug_length
0,2611,3465,35066,4,2005-11-29,1997-09-22,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
1,3210,3465,70224,4,1997-04-27,1994-01-17,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
2,4305,173,70358,1,\N,2015-10-19,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
3,1534,173,47007,1,2015-10-19,2004-10-04,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
4,1541,43,47014,1,2015-10-19,2004-10-04,313253,1621,2004-02-06 10:05:00-05,NaN,...,garry-breitkreuz-1,/debates/2004/2/6/garry-breitkreuz-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'today', ...",3


In [8]:
stop_words = set(stopwords.words('english'))

In [9]:
punctuation = '!"#$%&()*+,-./:;<=>@[]^_`{|}~' + "''"

In [10]:
pun_trans = str.maketrans(dict.fromkeys(punctuation, ' '))

In [11]:
ps = PorterStemmer() 

In [12]:
content_en_list = []  # append the content from each chunk df here 

In [13]:
def tokenize_column (row):
    soup = BeautifulSoup(row)
    text = soup.text.lower()
    text = text.translate(pun_trans)
    text = text.replace('\\n', ' ')
    text = text.replace('—', ' ')
    words = word_tokenize(text)
    return words

In [14]:
stop_words = set(stopwords.words('english'))

In [15]:
def remove_stopwords(word_tokens):
    tokens = [w for w in word_tokens if not w in stop_words]
    tokens = [t for t in tokens if not t.isdigit()]
    words = [ps.stem(w) for w in tokens]
    return words

In [16]:
small_df = df[:100]

In [17]:
df.columns

Index(['id', 'politician_id', 'riding_id', 'party_id', 'end_date',
       'start_date', 'id.1', 'document_id', 'time', 'h1_en', 'h2_en',
       'member_id', 'who_en', 'content_en', 'sequence_en', 'wordcount',
       'politician_id.1', 'procedural', 'h3_en', 'who_hocid', 'content_fr',
       'statement_type', 'written_question', 'source_id', 'who_context_en',
       'slug', 'urlcache', 'h1_fr', 'h2_fr', 'h3_fr', 'who_fr',
       'who_context_fr', 'wordcount_en', 'tokenized_content', 'slug_length'],
      dtype='object')

In [26]:
politicians = list(set(df.politician_id))

In [27]:
vocab_count_dict = defaultdict()

In [28]:
def add(k,v_list):
    if k in vocab_count_dict:
        for v in v_list:
            vocab_count_dict[k].add(v)
    else:
        for v in v_list:
            vocab_count_dict[k] = {v}

In [ ]:
%%time
max_tokens = 0
count = 0

for index, row in df.iterrows(): 
    soup = BeautifulSoup(row.content_en)
    text = soup.text.lower()
    text = text.translate(pun_trans)
    text = text.replace('\\n', ' ')
    text = text.replace('—', ' ')
    word_tokens = word_tokenize(text)
    tokens = [w for w in word_tokens if not w in stop_words]
    tokens = [t for t in tokens if not t.isdigit()]
    words = [ps.stem(w) for w in tokens]
    
    pid = row['politician_id']
    add(pid,words)
    
    if len(words)>max_tokens:
        max_tokens = len(words)
    if count%10000==0:
        print(count)
        print('updating id', row['politician_id'], words[:10])
    
    
    count+=1
#print(row.who_en, "gave the longest speech at", max_tokens, 'words')
    
        

0
updating id 3465 ['mr', 'speaker', 'rise', 'point', 'order', 'believ', 'would', 'find', 'unanim', 'consent']
10000
updating id 415 ['mr', 'speaker', 'ever', 'sinc', 'prime', 'minist', 'fled', 'davo', 'switzerland', 'announc']


In [24]:
vocab_count_dict[443]

{'?',
 'agre',
 'agreement',
 'associ',
 'believ',
 'bloc',
 'candid',
 'committe',
 'debt',
 'du',
 'far',
 'first',
 'gave',
 'gross',
 'hard',
 'hous',
 'islet',
 'issu',
 'kamouraska',
 'key',
 'l',
 'liber',
 'like',
 'local',
 'long',
 'lot',
 'loup',
 'make',
 'member',
 'montmagni',
 'motion',
 'mr',
 'oblig',
 'parti',
 'quebec',
 'question',
 'respect',
 'respons',
 'rivièr',
 'sens',
 'separ',
 'side',
 'speaker',
 'summer',
 'tell',
 'thank',
 'warm',
 'welcom',
 'work',
 'would',
 'île'}

In [25]:
vocab_count_dict[185]

{'?',
 'absurd',
 'actual',
 'add',
 'addit',
 'agreement',
 'altogeth',
 'american',
 'argument',
 'around',
 'aspect',
 'badli',
 'ban',
 'bean',
 'beanstalk',
 'becom',
 'behalf',
 'believ',
 'big',
 'breath',
 'canada',
 'canadian',
 'caucu',
 'cautionari',
 'centr',
 'chapter',
 'children',
 'cite',
 'claus',
 'clear',
 'colleagu',
 'commod',
 'compromis',
 'corpor',
 'could',
 'countri',
 'coupl',
 'cow',
 'deal',
 'deliber',
 'democrat',
 'deni',
 'disservic',
 'done',
 'econom',
 'either',
 'eloqu',
 'enter',
 'essenti',
 'even',
 'eventu',
 'exactli',
 'exampl',
 'expand',
 'fair',
 'famili',
 'feel',
 'felt',
 'find',
 'fit',
 'foreign',
 'forward',
 'free',
 'freer',
 'fresh',
 'fta',
 'fundament',
 'gall',
 'gasolin',
 'gave',
 'get',
 'give',
 'glare',
 'go',
 'halifax',
 'happi',
 'healthi',
 'hon',
 'hous',
 'industri',
 'interest',
 'interf',
 'issu',
 'jack',
 'jeopard',
 'joliett',
 'leav',
 'left',
 'like',
 'lose',
 'lost',
 'made',
 'make',
 'manufactur',
 'market'

In [35]:
len(politicians)

1051

In [36]:
politicians[56]

4156

In [ ]:
vocab_count_dict[4156]

In [38]:
len(vocab_count_dict[4156])

84078

In [ ]:
p_set = vocab_count_dict[4156]

In [ ]:
type(p_set)

In [ ]:
p_set

In [ ]:
p_set.remove('sacrifice.\\ntoday')

In [ ]:
p_set

In [ ]:
%%time
max_vocab = 0
vocab_politician = None
for key, p_set in vocab_count_dict.items():
    print(key, ": ", len(list(p_set)))
    if len(list(p_set))>max_vocab:
        max_vocab = len(list(p_set))
        vocab_politician = key

print(vocab_politician, 'has the biggest vocabulary at', max_vocab, 'words')

In [ ]:
list(vocab_count_dict.keys())